# Optimizing Hyper Parameters
This notebook belongs to a blog post I published on [Medium](https://medium.com/@timoboehm/how-to-optimized-98baec703593). Its purpose is to provide you with a quick and easy example how to apply __Grid Search__, __Randomized Search__, and __Bayesian Optimizaiton__ to hyper parameter optimization problems. I assume a basic familiarity with ```sklearn``` and other common packages. When it comes to Bayesian Optimization, I'll give you a basic idea on how ```hyperopt``` works.

<span style="color:blue">__Disclaimer:__ this notebook is about intution! In case of an actual data science project, please refer to the blog post for further links to in-depth material.</span>

## 0. Header

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import (train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score)
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier

I downloaded the dataset from [Kaggle](https://www.kaggle.com/kemical/kickstarter-projects#ks-projects-201801.csv). I restrict the analysis to the following columns from the original dataset:

- ```main_category```: as the name suggest, this indicates the larger category of a kickstarter project.
- ```category```: this is the more specific category. For instance, ```main_category``` could be _"Film"_ and ```category``` could be _"Shorts"_ 
- ```country```: exactly what the name suggests. Although most projects are based in the US (about 77%), there is also some variation that might help us to predict the result.
- ```usd_goal_real```: this feature already normalized the funding goal to USD.
- ```state```: this is the target variable we are looking for. For this notebook I only consider _"succesful"_ vs _"failed"_ and _"canceled"_. Observations with _"live"_ or _"undefined" are excluded from the analysis.

I sample 1000 observations to accelerate the calculation time.

In [2]:
# Read the data
data = pd.read_csv("data/kickstarter_projects.csv", usecols=["category", "main_category", "state",
                                                             "country", "usd_goal_real"])

# Filter based on the "state" column and sample 1000 observations from the remaining set.
data = data.loc[data.state.isin(["successful", "failed", "canceled"])].sample(1000, random_state=123)

# Look at five random examples from the dataset to understand the basic data structure.
data.sample(5)

,category,main_category,state,country,usd_goal_real
316713,Restaurants,Food,failed,US,25000.00
275116,Product Design,Design,failed,NL,40180.32
360753,Food,Food,failed,GB,37097.62
74036,Fiction,Publishing,canceled,US,6700.00
140396,Farms,Food,failed,US,35000.00


In [3]:
# Note: run this cell if you want to have a closer look at the five most frequent values for all categorical features.

#for c in data.columns:
#    if data[c].dtype == "object":
#        print(f"----------- {c} -----------\n")
#        print(f"{data[c].value_counts(normalize=True)[:5]}\n")

In [4]:
X = data.drop("state", axis=1)
X = pd.get_dummies(X)
y = data.state == "successful"

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, train_size=.8, random_state=42)

print(f"Number of observations in the training set: {X_train.shape[0]:{6}}")
print(f"Number of observations in the testing set:  {X_test.shape[0]:{6}}")
print(f"\nNumber of features available to predict outcome: {X_train.shape[1]}")
print(f"Proportion of successful kickstarter projects: {np.mean(y):.{2}}")

Number of observations in the training set:    800
Number of observations in the testing set:     200

Number of features available to predict outcome: 165
Proportion of successful kickstarter projects: 0.35


In [5]:
# Baseline Classifier to refer to in the next steps
clf = GradientBoostingClassifier(random_state=27)

# Dictionary to change the two hyper parameters we are focusing on here.
hyper_params = {
    "learning_rate": [.1],
    "max_features": [None]
}

In [6]:
# Function to evaluate the final models from each approach
def eval_final_model(classifier, x_test, y_test):
    y_pred = classifier.predict(x_test)
    return roc_auc_score(y_pred, y_test)

Use these values to decide on the range of the parameters and with how many possibilities you want to start your grid search.

In [7]:
learning_rate_min, learning_rate_max = .005, .2
max_features_min, max_features_max = 1, X_train.shape[1]

no_parameters = 5

print(f"Number of runs for GridSearch will be:            {no_parameters**2}")
print(f"Number of runs for RandomizedSearch will be:      {int(no_parameters**2/2)}")
print(f"Number of runs for Bayesian Optimization will be: {no_parameters**2}")

Number of runs for GridSearch will be:            25
Number of runs for RandomizedSearch will be:      12
Number of runs for Bayesian Optimization will be: 25


## Option 1: Grid Search
The grid is constructed with ```no_parameters``` values for each hyper parameter with linear distances between them:

In [8]:
hyper_params["learning_rate"] = np.linspace(learning_rate_min, learning_rate_max, no_parameters)
hyper_params["max_features"] = [int(n) for n in np.linspace(max_features_min, max_features_max, no_parameters)]

for k,v in hyper_params.items():
    print(f"{k}\n{v}")

learning_rate
[ 0.005    0.05375  0.1025   0.15125  0.2    ]
max_features
[1, 42, 83, 124, 165]


Based on this set of hyper parameters, we can now run sklearn's ```GridSearchCV``` implementation. I only defined the parameteres that are absolutely necessary. For more information, go [here](MISSING_LINK) <span style="color:red">ADD LINK </span>.

In [9]:
%%time
grid = GridSearchCV(clf,
                    hyper_params,
                    verbose=0)
grid.fit(X_train, y_train)

print(f"Score of the best model: {eval_final_model(grid.best_estimator_, X_test, y_test):.{3}}")

Score of the best model: 0.626
Wall time: 7.94 s


Applying grid search to our hyper parameter optimization problem, it took <span style="color:blue">about 7 seconds</span> for a score of <span style="color:blue">0.626</span> on the test set.

## Option 2: Randomized Search
We can now take advantage of defining a distribution of possible values instead of hard-coded options. I use scipy's ```randint``` function, that creates random integers within defined boundaries. The options for learning rates stay the same.

In [10]:
from scipy.stats import randint as randint

hyper_params["max_features"] = randint(max_features_min, max_features_max)
for k,v in hyper_params.items():
    print(f"{k}\n{v}")

learning_rate
[ 0.005    0.05375  0.1025   0.15125  0.2    ]
max_features


Again, we use sklearn but this time ```RandomizedSearchCV``` to implement randomized search. Remember that we use randomized search to save time, so we define the number of iterations (```n_iter```) as half of the original grid's size.

In [11]:
%%time
random = RandomizedSearchCV(clf,
                            hyper_params,
                            verbose=0,
                            n_iter=int(no_parameters**2 / 2))
random.fit(X_train, y_train)
print(f"Score of the best model: {eval_final_model(random.best_estimator_, X_test, y_test):.{3}}")

Score of the best model: 0.617
Wall time: 4.09 s


As expected, randomized search takes only about half the time than grid search (<span style="color:blue">about 3.5 seconds</span>) but its performance went down to <span style="color:blue">0.617</span>. That seems to be acceptable given that we spend only half the time with computing.

## Option 3: Bayesian Optimization

For a extensive example how to implenent ```hyperopt``` go [here](https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a). Bayesian optimization is conceptually harder than grid or randomized search and the necessary code reflects that. To get the intuition, think about the following steps:

- As in your everyday optimization problem, you have to define something to minimize. We used ```roc_auc_score```up to now, so we need to implement the _distance to a perfect score_ instead of the raw score. That's what ```min_score``` does.
- We also have to define our search space for hyper parameters. This time we use hyperopt's functions, but the idea stays the same.
- The other two parameters concern the actual algorithm to do the optimization and the number of allowed evaluations.

In [12]:
from hyperopt import hp   # Used to define the distribution for both parameters
from hyperopt import tpe  # Algorithm to apply
from hyperopt import fmin # We want to minimze the score_function

In [13]:
def min_score(params):
    params["max_features"] = int(params["max_features"])
    clf = GradientBoostingClassifier(**params)
    loss = np.mean(1 - cross_val_score(clf, X_train, y_train, scoring="roc_auc")) # We need to return something to minimize
    return loss

In [14]:
hyper_params = {
    'max_features': hp.quniform('max_features', max_features_min, max_features_max, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(learning_rate_min), np.log(learning_rate_max))
}

In [15]:
%%time
best = fmin(fn = min_score, space = hyper_params, algo = tpe.suggest, max_evals = no_parameters**2)
best["max_features"] = int(best["max_features"])
clf = GradientBoostingClassifier(**best)
clf.fit(X_train, y_train)
print(f"Score of the best model: {eval_final_model(clf, X_test, y_test):.{3}}")

Score of the best model: 0.635
Wall time: 6.72 s


We are back up to <span style="color:blue">about 6.5 seconds</span> but the performance rose to <span style="color:blue">0.635</span>. Given these results, there are two main take-aways:
1. Use randomized search instead of grid search.
2. Bayesian optimization is a powerful approach with a promising balance between calculating time and model performance.